In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv


In [2]:
visu_data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
visu_label = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [14]:
visu_data.head()
#visu_label.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [3]:
from pydoc import locate
X_train = pd.DataFrame(pd.read_csv('/kaggle/input/lish-moa/train_features.csv', nrows=5))
X_train.dtypes
types = [str(), str(), str(), str()]
for i in range(4, len(X_train.dtypes)):
    if X_train.dtypes[i].kind == 'f':
        types.append(float())


features = tf.data.experimental.CsvDataset('/kaggle/input/lish-moa/train_features.csv', record_defaults=types, header=True)


In [4]:
y = pd.DataFrame(pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv', nrows=5))
types = ['']
for i in range(1, len(y.dtypes)):
    types.append(float())

targets = tf.data.experimental.CsvDataset('/kaggle/input/lish-moa/train_targets_scored.csv', record_defaults=types, header=True)

In [5]:
dataset = tf.data.Dataset.zip((features, targets))

In [6]:
dataset_size = dataset.reduce(np.int64(0), lambda x, _:x+1).numpy()

In [7]:
train_dataset = dataset.take(0.7*dataset_size)
val_dataset = dataset.skip(0.7*dataset_size)
val_dataset = dataset.take(dataset_size - 0.7*dataset_size)

In [31]:
print(train_dataset.reduce(np.int64(0), lambda x, _:x+1).numpy())

16669


AttributeError: 'ZipDataset' object has no attribute 'variant_tensor'

In [8]:
def _preprocess_line(features, targets):
    # Pack the result into a dictionary
    features = dict(zip(visu_data.columns, features))
    #print(features)
    features.pop('sig_id')

    targets = tf.stack(targets[1:])
    print(targets)
    return features, targets

train_dataset = train_dataset.map(_preprocess_line).batch(5)

Tensor("stack:0", shape=(206,), dtype=float32)


In [60]:
for feature_batch, label_batch in train_dataset.take(1):
    print('First 5 features:', list(feature_batch.keys())[:5])
    print('A batch of cp_types:', feature_batch['cp_type'].numpy())
    print('A batch of cp_times:', feature_batch['cp_time'].numpy())
    print('A batch of targets:', label_batch.numpy() )

First 5 features: ['cp_type', 'cp_time', 'cp_dose', 'g-0', 'g-1']
A batch of cp_types: [b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp']
A batch of cp_times: [b'24' b'72' b'48' b'48' b'72']
A batch of targets: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
import autosklearn as ask
#ask.regression.AutoSklearnRegressor() for regression tasks
model = ask.classification.AutoSklearnClassifier(ensemble_size=10, #size of the end ensemble (minimum is 1)
                                                 time_left_for_this_task=120, #the number of seconds the process runs for
                                                 per_run_time_limit=30) #maximum seconds allocated per model
model.fit(train_dataset, y_train) #begin fitting the search model
print(model.sprint_statistics()) #print statistics for the search
y_predictions = model.predict(X_test) #get predictions from the model

In [48]:
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow_addons as tfa

model2 = Sequential([
    Input(shape=(875,)),
    tf.keras.layers.Dense(2048, input_shape=(875,), activation='relu'),
    tf.keras.layers.Dense(206, activation='sigmoid')
])



In [49]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
for features, label in train_dataset.take(1):
    model2.fit(features, label, batch_size=32, epochs=10)


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 2048)              1794048   
_________________________________________________________________
dense_33 (Dense)             (None, 206)               422094    
Total params: 2,216,142
Trainable params: 2,216,142
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t

In [55]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))
    for lab in label:
        print(lab)
show_batch(train_dataset)

cp_type             : [b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp']
cp_time             : [b'24' b'72' b'48' b'48' b'72']
cp_dose             : [b'D1' b'D1' b'D1' b'D1' b'D2']
g-0                 : [ 1.062   0.0743  0.628  -0.5138 -0.3254]
g-1                 : [ 0.5577  0.4087  0.5817 -0.2491 -0.4009]
g-2                 : [-0.2479  0.2991  1.554  -0.2656  0.97  ]
g-3                 : [-0.6208  0.0604 -0.0764  0.5288  0.6919]
g-4                 : [-0.1944  1.019  -0.0323  4.062   1.418 ]
g-5                 : [-1.012   0.5207  1.239  -0.8095 -0.8244]
g-6                 : [-1.022   0.2341  0.1715 -1.959  -0.28  ]
g-7                 : [-0.0326  0.3372  0.2155  0.1792 -0.1498]
g-8                 : [ 0.5548 -0.4047  0.0065 -0.1321 -0.8789]
g-9                 : [-0.0921  0.8507  1.23   -1.06    0.863 ]
g-10                : [ 1.183  -1.152  -0.4797 -0.8269 -0.2219]
g-11                : [ 0.153  -0.4201 -0.5631 -0.3584 -0.5121]
g-12                : [ 0.5574 -0.0958 -0.0366 -0.

g-435               : [-0.3804  0.0354 -1.56   -0.3784  0.    ]
g-436               : [ 0.3381  0.1407 -0.6829 -0.0766  1.184 ]
g-437               : [-0.4014  0.4225  0.8864  1.678  -0.5506]
g-438               : [-0.0093  0.6169 -0.146   0.2981 -0.5521]
g-439               : [-0.1907  0.1912 -0.8346 -2.068   0.1645]
g-440               : [ 0.7591 -0.7532 -0.1508 -0.3044  0.1631]
g-441               : [-0.212   0.1076 -0.6829 -0.2097 -0.2358]
g-442               : [ 1.06   -1.564   0.5365  1.048  -0.4004]
g-443               : [ 0.0995 -0.09    0.5518  0.395  -0.6745]
g-444               : [-0.4298 -0.0638  0.8081  1.38    0.6745]
g-445               : [ 0.9586  0.447  -0.5251  0.0449 -0.3815]
g-446               : [ 0.5436  0.2506 -0.0437  0.4697  0.9565]
g-447               : [ 0.8127 -1.127  -0.2788 -0.3887 -0.2797]
g-448               : [-0.1461  0.1764  0.7479 -0.506   0.0558]
g-449               : [ 0.3318 -0.6652 -0.0791  0.7361 -0.7133]
g-450               : [ 0.9664  0.5373  

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'items'

In [54]:
for features, label in train_dataset.take(1):
    print(label)


tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(5, 206), dtype=float32)
